Enhanced record minimum standard (ERMS) is the minimum standard of data enhancement for heritage places. The report of Heritage Places ERMS is done downstream, once the heritages places (HP) have been recorded in the database 

## Install

## Python libraries

In [ ]:
import pandas as pd
import numpy as np
import requests
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Environment

Clone [eamena-functions](https://github.com/eamena-project/eamena-functions) repo and import the script `erms.py`

ℹ️ Do not run locally

In [4]:
!rm /content/eamena-functions -R
!git clone https://github.com/eamena-project/eamena-functions.git
%cd /content/eamena-functions
from erms import erms

## Reference Data

Show the dataframe where the ERMS field are listes (`Yes` in the column `Enhanced record minimum standard`)

In [ ]:
df_listed = erms.erms_template()
# print(df_listed.to_markdown(index=False))

## Select the level of aggregation

See the documentation for 

In [ ]:
options=['level1', 'level2', 'level3']
radio_button = widgets.RadioButtons(
    options=options,
    description='Select an option:'
)
display(radio_button)

Show the aggregated ERMS dataframe. The column `value` has `1` for any given fields beloging to the ERMS standards

In [ ]:
df_erms = erms.erms_template_levels(radio_button = radio_button)
# print(df_erms.to_markdown(index=False))

Print only the ERMS standards

In [ ]:
print(df_erms.loc[df_erms['value'] == 1].to_markdown(index=False))

## Business data

### GeoJSON URL

Paste your GeoJSON URL

In [ ]:
GEOJSON_URL = "https://database.eamena.org/api/search/export_results?paging-filter=1&tiles=true&format=geojson&reportlink=false&precision=6&total=326&language=en&advanced-search=%5B%7B%22op%22%3A%22and%22%2C%2234cfea78-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22~%22%2C%22lang%22%3A%22en%22%2C%22val%22%3A%22Sistan%22%7D%2C%2234cfea87-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%22%22%7D%7D%2C%7B%22op%22%3A%22or%22%2C%2234cfea69-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%22%22%7D%2C%2234cfea73-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%22%22%7D%2C%2234cfea43-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%224ed99706-2d90-449a-9a70-700fc5326fb1%22%7D%2C%2234cfea5d-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%22%22%7D%2C%2234cfea95-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22~%22%2C%22lang%22%3A%22en%22%2C%22val%22%3A%22%22%7D%7D%5D&resource-type-filter=%5B%7B%22graphid%22%3A%2234cfe98e-c2c0-11ea-9026-02e7594ce0a0%22%2C%22name%22%3A%22Heritage%20Place%22%2C%22inverted%22%3Afalse%7D%5D&map-filter=%7B%22type%22%3A%22FeatureCollection%22%2C%22features%22%3A%5B%7B%22id%22%3A%22e84886109295dcb2d515f9ab792832bf%22%2C%22type%22%3A%22Feature%22%2C%22properties%22%3A%7B%22buffer%22%3A%7B%22width%22%3A10%2C%22unit%22%3A%22m%22%7D%2C%22inverted%22%3Afalse%7D%2C%22geometry%22%3A%7B%22coordinates%22%3A%5B%5B%5B61.5629662657594%2C31.341070427554456%5D%2C%5B61.39269902363566%2C31.226740239181964%5D%2C%5B61.52316353383432%2C30.977760218239922%5D%2C%5B61.773036239808164%2C30.92940344148805%5D%2C%5B61.89244443558445%2C31.037461248216815%5D%2C%5B61.933352798951745%2C31.22484931983834%5D%2C%5B61.5629662657594%2C31.341070427554456%5D%5D%5D%2C%22type%22%3A%22Polygon%22%7D%7D%5D%7D"

### Read

In [ ]:
hps = erms.db_query(GEOJSON_URL)
selected_hp = erms.hps_list(hps)
tot = str(len(selected_hp))
print(f"first of {tot} HPs: " + str(selected_hp[:5]))

### Subset

Optionaly you can subset the dataset of HP by GS (to avoid ultra-high number to display)

In [ ]:
# For any reason, if the current function goes to the erms.py file, the  'filtered_hp_gs' cannot be captured (cf. global, widgets, etc.)
def filter_hp_by_gs(hps, selected_hp):
    l_GridIDs = []
    for i in range(len(selected_hp)):
        l_GridIDs.append(hps['features'][i]['properties']['Grid ID'])
    l_GridIDs = list(set(l_GridIDs))
    l_GridIDs.sort()
    radio_button_1 = widgets.RadioButtons(
        options=l_GridIDs,
        description='Select a Grid Square:'
    )
    output = widgets.Output()

    def on_radio_button_change(change):
        selected_value = change.new
        global filtered_hp_gs
        with output:
            output.clear_output()
            filtered_hp_gs = erms.filter_dataframe(hps, selected_hp, selected_value)
            # Print or display the result
            print(filtered_hp_gs, end="")

    radio_button_1.observe(on_radio_button_change, names='value')
    display(radio_button_1, output)

filter_hp_by_gs(hps, selected_hp)
print(filtered_hp_gs, end="")

Subset the original GeoJSON on these HPs, overwrite variables `hps` and `selected_hp`

In [ ]:
hps = erms.hps_subset_by_gs(hps, filtered_hp_gs)
selected_hp = hps_subset_by_gs

### Plot

In [ ]:
dict_hps = erms.hps_dict(hps, selected_hp, df_listed, mylevel = radio_button.value)
erms.plot_spidergraphs(dict_hps, df_erms, mylevel = radio_button.value)